In [1]:
%matplotlib inline
import seaborn
import numpy, scipy
import librosa, librosa.display


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tempo Estimation

Tempo ([Wikipedia](https://en.wikipedia.org/wiki/Tempo)) refers to the speed of a musical piece. More precisely, tempo refers to the rate of the musical beat and is given by the reciprocal of the beat period. Tempo is often defined in units of beats per minute (BPM).

In classical music, common tempo markings include grave, largo, lento, adagio, andante, moderato, allegro, vivace, and presto. See [Basic tempo markings](https://en.wikipedia.org/wiki/Tempo#Basic_tempo_markings) for more.

Grave – (20–40 BPM)

Lento -  (40–45 BPM) 

Largo –   (45–50 BPM)

Larghetto –  (50–55 BPM)

Adagio – (55–65 BPM)

Adagietto - (65–69 BPM)

Andante – (73–77 BPM)

Moderato – (86–97 BPM)

Allegretto  (98–109 BPM)

Allegro – (109–132 BPM)

Vivo, or Vivace – (132–140 BPM)

Presto –  (168–177 BPM)

Prestissimo – 178+ BPM


# Run tempo comparison between two songs

Load two songs to compare

In [3]:
#import pro recording of the song
x1, sr1 = librosa.load('/content/drive/MyDrive/Music-moderation/Flowkey/BaselineDataset/WavFiles/nocturne_no2_adv.wav') # 117 bpm andante
#load the users recording
x2, sr2 = librosa.load('/content/drive/MyDrive/Music-moderation/Flowkey/BaselineDataset/WavFiles/nocturne_no2_pro.wav') # 123 bpm andante


In [4]:
def estimate_tempo(x,sr):
  #compute the onset envelope
  hop_length = 200 # samples per frame
  onset_env = librosa.onset.onset_strength(x, sr=sr, hop_length=hop_length, n_fft=2048)

  #compute the tempogram
  tempogram = librosa.feature.tempogram(onset_envelope=onset_env, sr=sr, hop_length=hop_length, win_length=400)
  #plot tempogram
  #librosa.display.specshow(tempogram, sr=sr, hop_length=hop_length, x_axis='time', y_axis='tempo')

  #estimate and return the global tempo in BPM
  tempo = librosa.beat.tempo(x, sr=sr)

  return int(tempo)


In [5]:
def tempo_marking(bpm):
    # select the tempo marking from the BPM range
    bpm_check = {
                    range(20, 39) : 'Grave',
                    range(40, 44) : 'Lento',
                    range(45, 49) : 'Largo',
                    range(50, 54) : 'Larghetto',
                    range(55, 64) : 'Adagio',
                    range(65, 69) : 'Adagietto',
                    range(73, 77) : 'Andante',
                    range(86, 97) : 'Moderato',
                    range(98, 108) : 'Allegretto',
                    range(109, 131) : 'Allegro',
                    range(132, 140) : 'Vivo or Vivace',
                    range(168, 177) : 'Presto',
                    range(178, 999) : 'Prestissimo'                
                    }

    for key in bpm_check:
      if bpm in key:
        return bpm_check[key]

Calculate and return tempo difference

In [6]:
pro_bpm = estimate_tempo(x1,sr1)
user_bpm = estimate_tempo(x2,sr2)

bpm_diff = numpy.absolute(user_bpm - pro_bpm)

if user_bpm < pro_bpm:
  print("You played", bpm_diff, "BPM too slowly.")
elif user_bpm > pro_bpm:
  print("You played", bpm_diff, "BPM to quickly.")
else:
  print("Your tempo is perfect")

print("Your tempo:", user_bpm, "BPM (",tempo_marking(user_bpm),")")
print("Pro tempo:", pro_bpm, "BPM (",tempo_marking(pro_bpm),")")
print("Error:", int(numpy.absolute(1-user_bpm/pro_bpm)*100),"%")

You played 6 BPM to quickly.
Your tempo: 123 BPM ( Allegro )
Pro tempo: 117 BPM ( Allegro )
Error: 5 %
